In [2]:
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 15.1 MB/s eta 0:00:00


In [3]:
pip install dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.38
    Uninstalling SQLAlchemy-2.0.38:
      Successfully uninstalled SQLAlchemy-2.0.38
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00


In [6]:
import random
import torch
import re
import torch.nn.functional as F
from datasets import Dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForMaskedLM
from docx import Document

# 读取 Word 文档
doc_path = "/content/bronze_cleaned.docx"
document = Document(doc_path)

sentences = []
for para in document.paragraphs:
    text = para.text.strip()
    if text:
        # 用正则表达式按 "，" 或 "。" 分割
        split_sentences = re.split(r"[，。]", text)
        # 过滤掉空句子
        sentences.extend([s.strip() for s in split_sentences if s.strip()])

# 过滤空句子
sentences = [s.strip() for s in sentences if len(s.strip()) > 0]
print(f"提取到 {len(sentences)} 句金文。")

def prepare_mixed_data_dynamic(sentence, tokenizer, max_length=128, mask_prob=0.15):
    encoded = tokenizer(sentence, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
    input_ids = encoded["input_ids"].squeeze(0)

    labels = input_ids.clone()

    # 计算需要 Mask 的位置
    probability_matrix = torch.full(input_ids.shape, mask_prob)
    masked_indices = torch.bernoulli(probability_matrix).bool()

    labels[~masked_indices] = -100
    input_ids[masked_indices] = tokenizer.mask_token_id  # 替换为 [MASK]

    return {
        "input_ids": input_ids.tolist(),
        "attention_mask": encoded["attention_mask"].squeeze(0).tolist(),
        "labels": labels.tolist(),
    }


def perturb_text(text, prob=0.1):
    chars = list(text)
    for i in range(len(chars)):
        if random.random() < prob:
            if random.random() < 0.3 and i < len(chars) - 1:  # 交换相邻字符
                chars[i], chars[i + 1] = chars[i + 1], chars[i]
            elif random.random() < 0.5:  # 删除字符
                chars[i] = ""
            else:  # 替换字符（这里用“□”表示缺失字符）
                chars[i] = "□"
    return "".join(chars)
# 先定义 perturbed_sentences
perturbed_sentences = []

# 遍历 sentences 并执行 perturbation
for s in sentences:
    perturbed = perturb_text(s)
    if len(perturbed.strip()) > 0:
        perturbed_sentences.append(perturbed)
    else:
        print(f"丢失句子: {s}")  # Debug 看看哪些句子丢了

print(f"扰动后保留的句子数: {len(perturbed_sentences)}")

def prepare_sop_data(sentence, next_sentence, tokenizer, max_length=128):
    encoded = tokenizer(sentence, next_sentence, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
    input_ids = encoded["input_ids"].squeeze(0).tolist()

    label = 1 if random.random() < 0.5 else 0  # 50% 保持顺序，50% 交换
    if label == 0:
        sentence, next_sentence = next_sentence, sentence  # 交换顺序

    return {
        "input_ids": input_ids,
        "attention_mask": encoded["attention_mask"].squeeze(0).tolist(),
        "sop_labels": label
    }

sentences_augmented = [perturb_text(s) for s in sentences]

# 加载 Tokenizer
model_path = "/content/drive/MyDrive/evahan/GujiRoBERTa_jian_fan"
tokenizer = AutoTokenizer.from_pretrained(model_path)
dataset = Dataset.from_list([prepare_mixed_data_dynamic(s, tokenizer) for s in sentences_augmented])
print(f"数据总数: {len(dataset)}")  # 检查总数是否正确

# 过滤掉 None 或异常数据
dataset = [data for data in dataset if data and len(data["input_ids"]) > 0]

# 再次检查
print(f"过滤后数据总数: {len(dataset)}")

class FocalLoss(torch.nn.Module):
    def __init__(self, gamma=2):
        super(FocalLoss, self).__init__()
        self.gamma = gamma

    def forward(self, logits, targets):
        ce_loss = F.cross_entropy(logits, targets, reduction="none")
        pt = torch.exp(-ce_loss)  # 计算正确分类的概率
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss  # 加强难分类样本的影响
        return focal_loss.mean()

class MLMandSOPTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        outputs = model(**inputs)

        # MLM Loss
        logits = outputs.logits
        labels = inputs["labels"]
        loss_mlm = F.cross_entropy(logits.view(-1, model.config.vocab_size), labels.view(-1), ignore_index=-100)

        # SOP Loss (确保数据集有 sop_labels)
        if "sop_labels" in inputs:
            sop_logits = outputs.logits[:, :2]  # 取前两个 logit 作为二分类输出
            sop_labels = inputs["sop_labels"]
            loss_sop = F.cross_entropy(sop_logits.view(-1, 2), sop_labels.view(-1))
        else:
            loss_sop = torch.tensor(0.0, device=logits.device)  # 避免 NoneType 问题

        loss = loss_mlm + 0.5 * loss_sop  # 组合损失

        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir="./GujiRoBERTa_finetuned",  # 训练结果保存位置
    per_device_train_batch_size=32,  # 根据显存调整 batch_size
    gradient_accumulation_steps=2,  # 4 个 batch 累计计算一次梯度
    num_train_epochs=10,  # 训练 5 轮
    learning_rate=5e-5,  # 适当调整学习率
    warmup_steps=50,  # 增加 warmup step，防止初始震荡
    weight_decay=0.01,  # 增加 L2 正则化
    fp16=True,
    logging_dir="./logs",  # 训练日志
    logging_steps=10,
    save_strategy="epoch",  # 每个 epoch 保存一次
    save_total_limit=2,  # 最多保存 2 个 checkpoint
)

model = AutoModelForMaskedLM.from_pretrained(model_path)

trainer = MLMandSOPTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

trainer.train()

model_save_path = "/content/drive/MyDrive/GujiRoBERTa_finetuned_bronze"
trainer.save_model(model_save_path)

提取到 5922 句金文。
丢失句子: 冊
丢失句子: 旅
丢失句子: 冉
丢失句子: 旦
扰动后保留的句子数: 5918
数据总数: 5922
过滤后数据总数: 5922


<ipython-input-6-7fcaa836c121>:147: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MLMandSOPTrainer.__init__`. Use `processing_class` instead.
  trainer = MLMandSOPTrainer(


Step,Training Loss
10,8.432900
20,1.274800
30,0.519600
40,0.401500
50,0.269000
60,0.260000
70,0.244000
80,0.225800
90,0.222800
100,0.190100


In [8]:
from transformers import pipeline, BertTokenizer # Import BertTokenizer

# Load the tokenizer from the original model path
tokenizer = BertTokenizer.from_pretrained(model_save_path)

# Initialize the pipeline with the correct tokenizer
fill_mask = pipeline("fill-mask", model=model_save_path, tokenizer=tokenizer) # Pass tokenizer instead of model

text = "唯王正[MASK]，初吉祥乙巳，□朱句之孫□亘□喪，王欲復師，擇吉金，自乍禾童，台樂賓客，"
predictions = fill_mask(text)

for pred in predictions:
    print(f"预测字: {pred['token_str']} (概率: {pred['score']:.4f})")

Device set to use cuda:0


预测字: 月 (概率: 1.0000)
预测字: 元 (概率: 0.0000)
预测字: 初 (概率: 0.0000)
预测字: 冬 (概率: 0.0000)
预测字: 陽 (概率: 0.0000)


In [12]:
from sklearn.metrics import precision_recall_fscore_support
import torch
from torch.utils.data import DataLoader

# 载入测试数据
test_dataset = Dataset.from_list([prepare_mixed_data_dynamic(s, tokenizer) for s in sentences[:100]])  # 取前100条作为测试集

# 自定义 collate_fn 确保 DataLoader 返回 batch 格式正确
def collate_fn(batch):
    batch_dict = {key: [d[key] for d in batch] for key in batch[0]}
    return {key: torch.tensor(val) for key, val in batch_dict.items()}

test_dataloader = DataLoader(test_dataset, batch_size=16, collate_fn=collate_fn)

# 载入训练好的模型
model = AutoModelForMaskedLM.from_pretrained(model_save_path)
model.eval()  # 设置为评估模式

all_preds, all_labels = [], []

# 遍历测试集
for batch in test_dataloader:
    batch = {k: v.to(model.device) for k, v in batch.items()}  # 确保 batch 数据都在 GPU/CPU 上
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    preds = torch.argmax(logits, dim=-1)  # 取最大值作为预测值

    # 只计算非 -100 的 label
    mask = labels != -100
    preds = preds[mask].cpu().numpy()
    labels = labels[mask].cpu().numpy()

    all_preds.extend(preds)
    all_labels.extend(labels)

# 计算 Precision, Recall, F1-score
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')

print(f"MLM 评估结果：")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

MLM 评估结果：
Precision: 0.2762, Recall: 0.2859, F1-score: 0.2738


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
from sklearn.metrics import precision_recall_fscore_support
import torch
from torch.utils.data import DataLoader

# 载入测试数据
test_dataset = Dataset.from_list([prepare_mixed_data_dynamic(s, tokenizer) for s in sentences[:100]])  # 取前100条作为测试集

# 自定义 collate_fn 确保 DataLoader 返回 batch 格式正确
def collate_fn(batch):
    batch_dict = {key: [d[key] for d in batch] for key in batch[0]}
    return {key: torch.tensor(val) for key, val in batch_dict.items()}

test_dataloader = DataLoader(test_dataset, batch_size=16, collate_fn=collate_fn)

# 载入训练好的模型
model = AutoModelForMaskedLM.from_pretrained(model_save_path)
model.eval()  # 设置为评估模式

all_correct, all_total = 0, 0  # 统计正确预测的 mask token 数量

# 遍历测试集
for batch in test_dataloader:
    batch = {k: v.to(model.device) for k, v in batch.items()}  # 确保 batch 数据都在 GPU/CPU 上
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # (batch_size, seq_len, vocab_size)

    # 计算 top-5 预测
    topk_preds = torch.topk(logits, k=5, dim=-1).indices  # (batch_size, seq_len, 10)

    # 只计算非 -100 的 label
    mask = labels != -100
    masked_labels = labels[mask]  # 真实 token
    masked_topk_preds = topk_preds[mask]  # 对应位置的 top-k 预测

    # 判断是否命中
    correct_mask = torch.any(masked_topk_preds == masked_labels.unsqueeze(-1), dim=-1)  # 至少有一个预测正确
    all_correct += correct_mask.sum().item()
    all_total += correct_mask.numel()

# 计算 Precision, Recall, F1-score
precision = all_correct / all_total if all_total > 0 else 0
recall = precision  # 这里 recall 与 precision 相同，因为预测的是单个 token
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print(f"MLM 评估结果（TOP-5）：")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")


MLM 评估结果（TOP-5）：
Precision: 0.9799, Recall: 0.9799, F1-score: 0.9799


In [14]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
from torch.utils.data import DataLoader

# 选择一个未经训练的 BERT，例如 'bert-base-chinese' 或其他适合古文的 BERT
untrained_model_name = "bert-base-chinese"
untrained_model = AutoModelForMaskedLM.from_pretrained(untrained_model_name)
untrained_tokenizer = AutoTokenizer.from_pretrained(untrained_model_name)

untrained_model.eval()  # 设置为评估模式

# 重新准备测试数据，确保适配未经训练的 BERT
test_dataset_untrained = Dataset.from_list([prepare_mixed_data_dynamic(s, untrained_tokenizer) for s in sentences[:100]])

def collate_fn(batch):
    batch_dict = {key: [d[key] for d in batch] for key in batch[0]}
    return {key: torch.tensor(val) for key, val in batch_dict.items()}

test_dataloader_untrained = DataLoader(test_dataset_untrained, batch_size=16, collate_fn=collate_fn)

# 统计正确预测的 mask token 数量
all_correct_untrained, all_total_untrained = 0, 0

# 遍历测试集
for batch in test_dataloader_untrained:
    batch = {k: v.to(untrained_model.device) for k, v in batch.items()}  # 确保 batch 数据都在 GPU/CPU 上
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]

    with torch.no_grad():
        outputs = untrained_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # (batch_size, seq_len, vocab_size)

    # 计算 top-10 预测
    topk_preds = torch.topk(logits, k=10, dim=-1).indices  # (batch_size, seq_len, 10)

    # 只计算非 -100 的 label
    mask = labels != -100
    masked_labels = labels[mask]  # 真实 token
    masked_topk_preds = topk_preds[mask]  # 对应位置的 top-k 预测

    # 判断是否命中
    correct_mask = torch.any(masked_topk_preds == masked_labels.unsqueeze(-1), dim=-1)  # 至少有一个预测正确
    all_correct_untrained += correct_mask.sum().item()
    all_total_untrained += correct_mask.numel()

# 计算 Precision, Recall, F1-score
precision_untrained = all_correct_untrained / all_total_untrained if all_total_untrained > 0 else 0
recall_untrained = precision_untrained
f1_untrained = 2 * precision_untrained * recall_untrained / (precision_untrained + recall_untrained) if precision_untrained + recall_untrained > 0 else 0

print(f"未经训练的 BERT 评估结果（TOP-10）：")
print(f"Precision: {precision_untrained:.4f}, Recall: {recall_untrained:.4f}, F1-score: {f1_untrained:.4f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

未经训练的 BERT 评估结果（TOP-10）：
Precision: 0.0037, Recall: 0.0037, F1-score: 0.0037


In [15]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
from torch.utils.data import DataLoader

# 载入未经微调的 GujiRoBERTa
untrained_model_path = "/content/drive/MyDrive/evahan/GujiRoBERTa_jian_fan"
untrained_model = AutoModelForMaskedLM.from_pretrained(untrained_model_path)  # 未微调的模型
untrained_tokenizer = AutoTokenizer.from_pretrained(untrained_model_path)

untrained_model.eval()  # 设置为评估模式

# 重新准备测试数据，确保适配 GujiRoBERTa
test_dataset_untrained = Dataset.from_list([prepare_mixed_data_dynamic(s, untrained_tokenizer) for s in sentences[:100]])

def collate_fn(batch):
    batch_dict = {key: [d[key] for d in batch] for key in batch[0]}
    return {key: torch.tensor(val) for key, val in batch_dict.items()}

test_dataloader_untrained = DataLoader(test_dataset_untrained, batch_size=16, collate_fn=collate_fn)

# 统计正确预测的 mask token 数量
all_correct_untrained, all_total_untrained = 0, 0

# 遍历测试集
for batch in test_dataloader_untrained:
    batch = {k: v.to(untrained_model.device) for k, v in batch.items()}  # 确保 batch 数据都在 GPU/CPU 上
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]

    with torch.no_grad():
        outputs = untrained_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # (batch_size, seq_len, vocab_size)

    # 计算 top-10 预测
    topk_preds = torch.topk(logits, k=10, dim=-1).indices  # (batch_size, seq_len, 10)

    # 只计算非 -100 的 label
    mask = labels != -100
    masked_labels = labels[mask]  # 真实 token
    masked_topk_preds = topk_preds[mask]  # 对应位置的 top-k 预测

    # 判断是否命中
    correct_mask = torch.any(masked_topk_preds == masked_labels.unsqueeze(-1), dim=-1)  # 至少有一个预测正确
    all_correct_untrained += correct_mask.sum().item()
    all_total_untrained += correct_mask.numel()

# 计算 Precision, Recall, F1-score
precision_untrained = all_correct_untrained / all_total_untrained if all_total_untrained > 0 else 0
recall_untrained = precision_untrained
f1_untrained = 2 * precision_untrained * recall_untrained / (precision_untrained + recall_untrained) if precision_untrained + recall_untrained > 0 else 0

print(f"未经微调的 GujiRoBERTa 评估结果（TOP-10）：")
print(f"Precision: {precision_untrained:.4f}, Recall: {recall_untrained:.4f}, F1-score: {f1_untrained:.4f}")


未经微调的 GujiRoBERTa 评估结果（TOP-10）：
Precision: 0.0435, Recall: 0.0435, F1-score: 0.0435
